In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(ConvBlock, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )
    
    def forward(self, x):
        return self.conv(x)

class UNet(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(UNet, self).__init__()
        
        # Encoder
        self.enc1 = ConvBlock(in_channels, 64)
        self.enc2 = ConvBlock(64, 128)
        self.enc3 = ConvBlock(128, 256)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        
        # Bottleneck
        self.bottleneck = ConvBlock(256, 512)
        
        # Decoder
        self.upconv3 = nn.ConvTranspose2d(512, 256, kernel_size=2, stride=2)
        self.dec3 = ConvBlock(512, 256)
        self.upconv2 = nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2)
        self.dec2 = ConvBlock(256, 128)
        self.upconv1 = nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2)
        self.dec1 = ConvBlock(128, 64)
        
        # Output layer
        self.out_conv = nn.Conv2d(64, out_channels, kernel_size=1)
    
    def forward(self, x):
        # Encoder
        enc1 = self.enc1(x)
        enc2 = self.enc2(self.pool(enc1))
        enc3 = self.enc3(self.pool(enc2))
        
        # Bottleneck
        bottleneck = self.bottleneck(self.pool(enc3))
        
        # Decoder
        dec3 = self.upconv3(bottleneck)
        dec3 = self.dec3(torch.cat([dec3, enc3], dim=1))
        dec2 = self.upconv2(dec3)
        dec2 = self.dec2(torch.cat([dec2, enc2], dim=1))
        dec1 = self.upconv1(dec2)
        dec1 = self.dec1(torch.cat([dec1, enc1], dim=1))
        
        # Output
        return self.out_conv(dec1)

# Instantiate the model
model = UNet(in_channels=1, out_channels=1)  


In [3]:
x = torch.randn(1, 1, 128, 128)  # Batch size 1, single-channel, 128x128 input
output = model(x)
print(output.shape)  


torch.Size([1, 1, 128, 128])


In [4]:
def count_parameters_and_size(model):
    # Count total parameters
    total_params = sum(p.numel() for p in model.parameters())

    # Count trainable parameters
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

    # Calculate model size in MB
    # Each parameter is typically stored in float32 format (4 bytes)
    model_size_bytes = total_params * 4  # 4 bytes per parameter
    model_size_mb = model_size_bytes / (1024 * 1024)  # Convert to MB

    return {
        'total_parameters': total_params,
        'trainable_parameters': trainable_params,
        'model_size_mb': model_size_mb
    }

# Get model statistics
stats = count_parameters_and_size(model)

# Print detailed information
print(f"Total Parameters: {stats['total_parameters']:,}")
print(f"Trainable Parameters: {stats['trainable_parameters']:,}")
print(f"Model Size: {stats['model_size_mb']:.2f} MB")

# For more detailed parameter count by layer
def print_model_parameters(model):
    print("\nDetailed parameter count by layer:")
    print("-" * 80)
    print(f"{'Layer':<40} {'Parameters':>12} {'Size (MB)':>12}")
    print("-" * 80)

    for name, parameter in model.named_parameters():
        params = parameter.numel()
        size_mb = (params * 4) / (1024 * 1024)  # size in MB
        print(f"{name:<40} {params:>12,} {size_mb:>12.2f}")
print_model_parameters(model)


Total Parameters: 7,701,825
Trainable Parameters: 7,701,825
Model Size: 29.38 MB

Detailed parameter count by layer:
--------------------------------------------------------------------------------
Layer                                      Parameters    Size (MB)
--------------------------------------------------------------------------------
enc1.conv.0.weight                                576         0.00
enc1.conv.0.bias                                   64         0.00
enc1.conv.1.weight                                 64         0.00
enc1.conv.1.bias                                   64         0.00
enc1.conv.3.weight                             36,864         0.14
enc1.conv.3.bias                                   64         0.00
enc1.conv.4.weight                                 64         0.00
enc1.conv.4.bias                                   64         0.00
enc2.conv.0.weight                             73,728         0.28
enc2.conv.0.bias                                  1

In [6]:
class ModelStatistics:
    def __init__(self, model: nn.Module):
        self.model = model

    def count_parameters_and_size(self):
        # Count total parameters
        total_params = sum(p.numel() for p in self.model.parameters())

        # Count trainable parameters
        trainable_params = sum(p.numel() for p in self.model.parameters() if p.requires_grad)
        # Calculate model size in MB
        # Each parameter is typically stored in float32 format (4 bytes)
        model_size_bytes = total_params * 4  # 4 bytes per parameter
        model_size_mb = model_size_bytes / (1024 * 1024)  # Convert to MB

        return {
            'total_parameters': total_params,
            'trainable_parameters': trainable_params,
            'model_size_mb': model_size_mb
        }

    def print_model_parameters(self):
        print("\nDetailed parameter count by layer:")
        print("-" * 80)
        print(f"{'Layer':<40} {'Parameters':>12} {'Size (MB)':>12}")
        print("-" * 80)

        for name, parameter in self.model.named_parameters():
            params = parameter.numel()
            size_mb = (params * 4) / (1024 * 1024)  # size in MB
            print(f"{name:<40} {params:>12,} {size_mb:>12.2f}")

    def print_statistics(self):
        stats = self.count_parameters_and_size()
        print(f"Total Parameters: {stats['total_parameters']:,}")
        print(f"Trainable Parameters: {stats['trainable_parameters']:,}")
        print(f"Model Size: {stats['model_size_mb']:.2f} MB")
        self.print_model_parameters()

In [7]:
model_stats = ModelStatistics(model)
model_stats.print_statistics()

Total Parameters: 7,701,825
Trainable Parameters: 7,701,825
Model Size: 29.38 MB

Detailed parameter count by layer:
--------------------------------------------------------------------------------
Layer                                      Parameters    Size (MB)
--------------------------------------------------------------------------------
enc1.conv.0.weight                                576         0.00
enc1.conv.0.bias                                   64         0.00
enc1.conv.1.weight                                 64         0.00
enc1.conv.1.bias                                   64         0.00
enc1.conv.3.weight                             36,864         0.14
enc1.conv.3.bias                                   64         0.00
enc1.conv.4.weight                                 64         0.00
enc1.conv.4.bias                                   64         0.00
enc2.conv.0.weight                             73,728         0.28
enc2.conv.0.bias                                  1